In [1]:
from guizero import App, Text, Picture, PushButton
import time
import random
from pathlib import Path
from PIL import Image

DINO_IMAGE = Path("dino.png")
if not DINO_IMAGE.exists():
    raise FileNotFoundError(f"Error: {DINO_IMAGE.resolve()} not found! Ensure the image is in the same directory.")

BLANK_IMAGE = Path("blank.png")
if not BLANK_IMAGE.exists():
    blank_image = Image.new('RGB', (100, 100), color=(255, 255, 255))
    blank_image.save(BLANK_IMAGE)

reaction_times = []
start_time = 0
rounds_played = 0
TOTAL_ROUNDS = 10
game_in_progress = False

def start_game():
    global reaction_times, rounds_played, game_in_progress
    reaction_times = []
    rounds_played = 0
    game_in_progress = True
    result.value = ""
    instruction.value = "Wait for the dinosaur..."
    start_button.disable()
    dino.image = str(BLANK_IMAGE.resolve())
    play_round()

def play_round():
    global start_time, rounds_played, game_in_progress
    if not game_in_progress:
        return

    if rounds_played < TOTAL_ROUNDS:
        instruction.value = "Wait for the dinosaur..."
        dino.image = str(BLANK_IMAGE.resolve())
        delay = random.randint(2000, 5000)
        app.after(delay, show_dinosaur)
    else:
        show_results()

def show_dinosaur():
    global start_time, rounds_played, game_in_progress
    if not game_in_progress:
        return

    x = random.randint(50, app.width - 150)
    y = random.randint(50, app.height - 150)
    dino.image = str(DINO_IMAGE.resolve())
    dino.tk.place(x=x, y=y)

    instruction.value = "CLICK THE DINO!"
    start_time = time.time()
    dino.when_clicked = check_reaction 

def check_reaction(event):
    global rounds_played, game_in_progress
    if not game_in_progress:
        return

    reaction_time = time.time() - start_time
    reaction_times.append(reaction_time)
    rounds_played += 1

    result.value = f"Your reaction time: {reaction_time:.3f} seconds"

    if reaction_time < 0.5:
        result.text_color = "green"
        result.value += "\nExcellent reflexes!"
    elif reaction_time < 0.8:
        result.text_color = "blue"
        result.value += "\nGood job!"
    else:
        result.text_color = "red"
        result.value += "\nTry to be faster!"

    app.after(1000, play_round)
    dino.when_clicked = None  

def show_results():
    global game_in_progress
    avg_time = sum(reaction_times) / len(reaction_times)
    instruction.value = "Game Over!"
    result.value = f"Average Reaction Time: {avg_time:.3f} seconds"
    result.text_color = "black"
    start_button.enable()
    dino.image = str(BLANK_IMAGE.resolve())
    game_in_progress = False
    dino.when_clicked = None 


app = App(title="Dino Reaction Game", width=500, height=400, layout="auto")

instruction = Text(app, text="Press Start to Begin", size=12)
dino = Picture(app, image=str(BLANK_IMAGE.resolve()))
result = Text(app, text="", size=14)
start_button = PushButton(app, text="Start New Game", command=start_game, width=15, height=2)

start_button.tk.place(relx=0.5, rely=0.95, anchor="center")


app.display()